In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/images/test.jpg
/kaggle/input/sample-images/pot4.jpg
/kaggle/input/sample-images/pot3.jpg
/kaggle/input/sample-images/potf.jpg
/kaggle/input/sample-images/pot2.jpg
/kaggle/input/pd-jpg/pd.jpg


In [2]:
pip uninstall open3d -y

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install open3d

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.0 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
from PIL import Image
import open3d as o3d

# Load the 2D images
images = []
for i in range(10):
    image = Image.open("/kaggle/input/pd-jpg/pd.jpg".format(i))
    image = image.resize((256, 256))
    images.append(np.array(image))

# Convert the 2D images to a point cloud
point_cloud = o3d.geometry.PointCloud()
for i in range(10):
    img = images[i]
    h, w, _ = img.shape
    img_points = np.column_stack(np.where(np.any(img > 0, axis=-1)))
    img_points[:, [0, 1]] = img_points[:, [1, 0]]  
    depth_values = img[img_points[:, 0], img_points[:, 1], 0]  # Use the depth from the red channel (assuming it's the depth map)
    points = np.column_stack((img_points, depth_values))
    points[:, :2] = points[:, :2] - np.array([w / 2, h / 2])  # Centering the points
    pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(points))

    # Create a DBSCAN object
    eps = 0.1  
    min_points = 50 
    labels = np.array(pcd.cluster_dbscan(eps=eps, min_points=min_points, print_progress=True))

    # Assign the labels to the points
    pcd.colors = o3d.utility.Vector3dVector(np.random.rand(len(labels), 3))
    for i in range(len(labels)):
        pcd.colors[i] = pcd.colors[labels[i]]

    # Add the clustered point cloud to the overall point cloud
    point_cloud += pcd

# Save the point cloud
o3d.io.write_point_cloud("/kaggle/working/point_cloud.ply", point_cloud)


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%
Precompute neighbors.[========================================] 100%


True

In [5]:
import open3d as o3d

# Load the point cloud from a PLY file
pcd = o3d.io.read_point_cloud("/kaggle/working/point_cloud.ply")

# Compute normals for the point cloud
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Reconstruct the mesh using Screened Poisson Surface Reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9, width=0, scale=1.1, linear_fit=True)

# Save the mesh to a PLY file
o3d.io.write_triangle_mesh("/kaggle/working/output_screened_poisson.ply", mesh)


True

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import open3d as o3d
import numpy as np

# Define a more complex CNN for 3D shape completion/generation
class ImprovedMeshGeneratorCNN(nn.Module):
    def __init__(self, num_points):
        super(ImprovedMeshGeneratorCNN, self).__init__()
        self.num_points = num_points
        self.conv1 = nn.Conv1d(3, 64, kernel_size=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=1)
        self.conv3 = nn.Conv1d(128, 256, kernel_size=1)
        self.fc1 = nn.Linear(256, 512)
        self.fc2 = nn.Linear(512, 3 * self.num_points)

    def forward(self, input_cloud):
        x = self.conv1(input_cloud)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.conv3(x)
        x = torch.max(x, dim=2)[0]  
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        output_mesh = x.view(-1, 3, self.num_points)

        return output_mesh

# Load the mesh from a .ply file
mesh = o3d.io.read_triangle_mesh("/kaggle/working/output_screened_poisson.ply")

# Convert the mesh to a point cloud
point_cloud = np.asarray(mesh.vertices).T  

num_points = point_cloud.shape[1]

# Create a more complex MeshGeneratorCNN model
improved_mesh_generator_cnn = ImprovedMeshGeneratorCNN(num_points)

# Convert the input point cloud to a PyTorch tensor
input_cloud = torch.tensor(point_cloud, dtype=torch.float32).unsqueeze(0)

# Specify the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model and input data to the device
improved_mesh_generator_cnn.to(device)
input_cloud = input_cloud.to(device)

# Use a more sophisticated optimizer and scheduler
optimizer = optim.Adam(improved_mesh_generator_cnn.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.5)

# Specify an appropriate loss function 
loss_function = nn.MSELoss()

# Training loop
num_epochs = 50  
for epoch in range(num_epochs):
    improved_mesh_generator_cnn.train()

    # Forward pass
    output_mesh = improved_mesh_generator_cnn(input_cloud)

    # Compute the loss
    loss = loss_function(output_mesh, input_cloud)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Adjust learning rate
    scheduler.step()

    # Print training progress
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

# Convert the output mesh from PyTorch tensor to NumPy array
generated_mesh_np = output_mesh.detach().cpu().numpy().squeeze().transpose()

# Create a new Open3D TriangleMesh from the generated vertices and faces
generated_mesh = o3d.geometry.TriangleMesh()
generated_mesh.vertices = o3d.utility.Vector3dVector(generated_mesh_np)
generated_mesh.triangles = o3d.utility.Vector3iVector(mesh.triangles)

# Save the generated mesh to an .obj file
o3d.io.write_triangle_mesh("/kaggle/working/improved_output_generated_cnn.obj", generated_mesh)


Epoch 1/50, Loss: 15378.078125
Epoch 2/50, Loss: 15185.2587890625
Epoch 3/50, Loss: 14802.19921875
Epoch 4/50, Loss: 14143.34375
Epoch 5/50, Loss: 13165.0224609375
Epoch 6/50, Loss: 11837.8642578125
Epoch 7/50, Loss: 10161.837890625
Epoch 8/50, Loss: 8205.3974609375
Epoch 9/50, Loss: 6147.3515625
Epoch 10/50, Loss: 5158.875
Epoch 11/50, Loss: 4249.126953125
Epoch 12/50, Loss: 3479.06689453125
Epoch 13/50, Loss: 2904.34326171875
Epoch 14/50, Loss: 2541.999267578125
Epoch 15/50, Loss: 2319.236083984375
Epoch 16/50, Loss: 2090.033447265625
Epoch 17/50, Loss: 1761.7037353515625
Epoch 18/50, Loss: 1543.259033203125
Epoch 19/50, Loss: 1289.1917724609375
Epoch 20/50, Loss: 1031.8712158203125
Epoch 21/50, Loss: 797.4583129882812
Epoch 22/50, Loss: 602.6226806640625
Epoch 23/50, Loss: 454.6861267089844
Epoch 24/50, Loss: 352.8699645996094
Epoch 25/50, Loss: 290.53466796875
Epoch 26/50, Loss: 270.80657958984375
Epoch 27/50, Loss: 255.91383361816406
Epoch 28/50, Loss: 243.87283325195312
Epoch 29/

True